In [9]:
!pip install openai pymupdf numpy
!pip install docling
!pip install docling-core
!pip install docling-parse
import fitz
import os
import numpy as np
from openai import OpenAI
from google.colab import files
from docling_core.types.doc.page import TextCellUnit
from docling_parse.pdf_parser import DoclingPdfParser



In [10]:
print("Начало загрузки PDF...")
uploaded = files.upload()
pdf_filename = list(uploaded.keys())[0]
pdf_path = pdf_filename
print(f"✅ Файл '{pdf_filename}' успешно загружен.")

Начало загрузки PDF...


Saving test.pdf to test.pdf
✅ Файл 'test.pdf' успешно загружен.


In [11]:
def parse_pdf_document_docling(file_path: str) -> str:


    try:
        parser = DoclingPdfParser()
        pdf_doc = parser.load(path_or_stream=file_path)

        pages_text = []

        for page_no, page in pdf_doc.iterate_pages():

            words = [cell.text for cell in page.iterate_cells(unit_type=TextCellUnit.WORD)]
            line = " ".join(words)
            line = " ".join(line.split()).strip()
            pages_text.append(line)

        return "\n\n".join(pages_text)

    except Exception as e:
        print(f"Ошибка при чтении PDF через docling-parse: {e}")
        return ""
document_text = parse_pdf_document_docling(pdf_path)
print(f"Извлечено {len(document_text)} символов.")
print(document_text[:500])

Извлечено 399667 символов.
vGate R2 Руководство администратора Установка и эксплуатация RU.88338853.501410.012 91 2

© Компания "Код Безопасности", 2025. Все права защищены. Все авторские права на эксплуатационную документацию защищены. Этот документ входит в комплект поставки изделия. На него распространяются все условия лицензионного соглашения. Без специального письменного разрешения компании "Код Безопасности" этот документ или его часть в печатном или электронном виде не могут быть подвергнуты копированию и передаче 


In [13]:
def create_sliding_window_chunks(document_text, max_chars=1000, stride_overlap=200):
    corpus_fragment = []
    stride = max_chars - stride_overlap
    if stride <= 0:
        raise ValueError("Размер перекрытия должен быть меньше размера фрагмента")
    for start_index in range(0, len(document_text), stride):
        end_index = min(start_index + max_chars, len(document_text))
        fragment = document_text[start_index:end_index]
        corpus_fragment.append(fragment)

        if end_index == len(document_text):
            break
    return corpus_fragment
corpus_fragment = create_sliding_window_chunks(document_text,max_chars=500, stride_overlap=100)
print(f"Документ разбит на {len(corpus_fragment)} фрагментов для индексации")

Документ разбит на 999 фрагментов для индексации


In [14]:
os.environ["OPENAI_API_KEY"] = "v1.CmMKHHN0YXRpY2tleS1lMDBra2Y1cjd2bWowcG14dDYSIXNlcnZpY2VhY2NvdW50LWUwMGswNDZjYmQxendkY2RtOTILCLb-nMkGEJu5-ng6DAi1gbWUBxDA9onSAkACWgNlMDA.AAAAAAAAAAHYGVbgcc1d_gtAolYARmNu3putzcCoLDmiDkG6HBf9Y1XUZEOVlRqeFbCDzM4_getsSMIBS39tYn1kP9XPSHcE"
client = OpenAI(
    base_url="https://api.studio.nebius.com/v1/",
    api_key=os.getenv("OPENAI_API_KEY")
)
def create_embeddings(text_list, model="BAAI/bge-multilingual-gemma2"):
    response = client.embeddings.create(input=text_list, model=model)
    return response.data
print("Генерация эмбендингов для всего корпуса (требуется время)")
fragment_embeddings = create_embeddings(corpus_fragment)
print(f" Индексация завершена. Получено {len(fragment_embeddings)} векторов")

Генерация эмбендингов для всего корпуса (требуется время)
 Индексация завершена. Получено 999 векторов


In [15]:
def calculate_cosine_similarity(vec1, vec2):
    """Вычисляет косинусное сходство между двумя векторами NumPy."""
    # np.dot(vec1, vec2) - скалярное произведение
    # np.linalg.norm(vec) - Евклидова норма (длина) вектора
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

def retrieve_context(user_query, corpus_fragment, fragment_embeddings, k_neighbors=3):
    """Находит топ-K фрагментов корпуса, наиболее близких к запросу по смыслу.

    :param k_neighbors: Количество самых релевантных фрагментов для извлечения.
    :return: Список из K фрагментов текста.
    """
    # 1. Векторизация запроса
    query_embedding_response = client.embeddings.create(
        input=[user_query],
        model="BAAI/bge-multilingual-gemma2"
    )
    # Преобразуем список чисел в вектор NumPy для расчетов
    query_embedding = np.array(query_embedding_response.data[0].embedding)

    # 2. Расчет сходства
    similarity_scores = []
    for i, emb_data in enumerate(fragment_embeddings):
        fragment_vector = np.array(emb_data.embedding)
        score = calculate_cosine_similarity(query_embedding, fragment_vector)
        similarity_scores.append((i, score))

    # 3. Сортировка и выбор топ-K
    # Сортируем по убыванию (чем выше score, тем лучше)
    similarity_scores.sort(key=lambda x: x[1], reverse=True)
    top_k_indices = [index for index, score in similarity_scores[:k_neighbors]]

    return [corpus_fragment[i] for i in top_k_indices]

In [16]:
def generate_augmented_response(query, context_fragments, model_name="Qwen/Qwen3-32B"):
    """Формирует промпт из контекста и запроса и вызывает LLM для ответа."""

    system_prompt = "Вы — экспертный ассистент по работе с документами. Ваш ответ должен быть строго основан на предоставленном КОНТЕКСТЕ. Если в КОНТЕКСТЕ нет информации для полного ответа, вы должны сообщить, что информация не найдена, не выдумывая фактов."

    # Объединяем найденные фрагменты в единый, легко читаемый контекст
    context_text = "\n\n--- РЕЛЕВАНТНЫЙ ФРАГМЕНТ ---\n\n".join(context_fragments)

    full_user_message = f"""
КОНТЕКСТ ДЛЯ ОТВЕТА:
{context_text}

ВОПРОС ПОЛЬЗОВАТЕЛЯ:
{query}
"""

    response = client.chat.completions.create(
        model=model_name,
        temperature=0, # Обеспечивает фактическую точность и снижает "галлюцинации"
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": full_user_message}
        ]
    )

    return response.choices[0].message.content

In [17]:
# Интерактивный ввод вопроса
user_query = input("Введите ваш вопрос по загруженному документу: ")

print("\n" + "*"*40)
print("🔍 Этап 1: Извлечение контекста (Retrieval)")

# Извлекаем 3 самых релевантных фрагмента
top_k_fragments = retrieve_context(user_query, corpus_fragment, fragment_embeddings, k_neighbors=3)

# Выводим найденные фрагменты для проверки прозрачности работы
for i, chunk in enumerate(top_k_fragments):
    print(f"\n--- Найденный фрагмент {i+1} ---\n{chunk[:200]}...") # Выводим только первые 200 символов

print("\n" + "*"*40)
print("🤖 Этап 2: Генерация ответа (Augmentation)")

# Генерируем финальный ответ
final_answer = generate_augmented_response(user_query, top_k_fragments)

print("\n" + "="*60)
print("ФИНАЛЬНЫЙ ОТВЕТ СИСТЕМЫ RAG:")
print("="*60)
print(final_answer)
print("="*60)

Введите ваш вопрос по загруженному документу: Как установить vGate?

****************************************
🔍 Этап 1: Извлечение контекста (Retrieval)

--- Найденный фрагмент 1 ---
а экране появится диалог с перечнем программного обеспечения, содержащегося на установочном дис- ке. 3. Активируйте ссылку "Клиент vGate". Совет. Для выполнения установки этого продукта можно также за...

--- Найденный фрагмент 2 ---
мый сервер может быть недоступен. План установки Развертывание vGate рекомендуется проводить в следующем порядке: № Шаг установки Особенности Описание 1 Выбор архитектуры vGate Перед настройкой сети и...

--- Найденный фрагмент 3 ---
з каталога \vGate R2\ на установочном компакт-диске. • Активируйте в Панели управления компонент "Программы и компоненты". Выберите в списке установленных программ элемент "Клиент vGate 5.1" и нажмите...

****************************************
🤖 Этап 2: Генерация ответа (Augmentation)

ФИНАЛЬНЫЙ ОТВЕТ СИСТЕМЫ RAG:
<think>
Хорошо, пользователь сп